In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2024-09-24 15:46:19.192122: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
pixels_Usage = pd.read_csv('fer2013.csv').iloc[:,1:]

In [18]:
pixels_Usage.head()

,pixels,Usage
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [22]:
labels = pd.read_csv('fer2013new.csv').iloc[:,2:]

In [25]:
labels.head()

,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,4,0,0,1,3,2,0,0,0,0
1,6,0,1,1,0,0,0,0,2,0
2,5,0,0,3,1,0,0,0,1,0
3,4,0,0,4,1,0,0,0,1,0
4,9,0,0,1,0,0,0,0,0,0


In [28]:
df = pd.concat([pixels_Usage,labels],axis=1)

In [29]:
df.head()

,pixels,Usage,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,4,0,0,1,3,2,0,0,0,0
1,151 150 147 155 148 133 111 140 170 174 182 15...,Training,6,0,1,1,0,0,0,0,2,0
2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,5,0,0,3,1,0,0,0,1,0
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,4,0,0,4,1,0,0,0,1,0
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,9,0,0,1,0,0,0,0,0,0


In [36]:
df[df['NF']>0].groupby('Usage').size()  # report count of images with NF votes >0 based on the dataset they belong to (private=test,public=validation)

Usage
PrivateTest     72
PublicTest      20
Training       257
dtype: int64

In [39]:
df.groupby('Usage').size()  # as we can see, they're a very low percentage with respect to the total amount, so we'll just delete them later

Usage
PrivateTest     3589
PublicTest      3589
Training       28709
dtype: int64

In [47]:
# check for rows with NaNs or null values; in this case, there are none
print(df[df.isna().any(axis=1)])
print(df[df.isnull().any(axis=1)])  

Empty DataFrame
Columns: [pixels, Usage, neutral, happiness, surprise, sadness, anger, disgust, fear, contempt, unknown, NF]
Index: []
Empty DataFrame
Columns: [pixels, Usage, neutral, happiness, surprise, sadness, anger, disgust, fear, contempt, unknown, NF]
Index: []


In [51]:
df_filtered = df[df['NF']==0]
print(df_filtered[df_filtered['NF']>0])
print(df.shape,df_filtered.shape) # the filter was successful

Empty DataFrame
Columns: [pixels, Usage, neutral, happiness, surprise, sadness, anger, disgust, fear, contempt, unknown, NF]
Index: []
(35887, 12) (35538, 12)


In [54]:
df_filtered.drop(columns=['NF'],inplace=True)

/var/folders/2r/kmz9wfbn6vn03_888ghbrcd00000gn/T/ipykernel_2334/726027918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['NF'],inplace=True)


In [56]:
print(df_filtered.shape)
df_filtered.head()

(35538, 11)


,pixels,Usage,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,4,0,0,1,3,2,0,0,0
1,151 150 147 155 148 133 111 140 170 174 182 15...,Training,6,0,1,1,0,0,0,0,2
2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,5,0,0,3,1,0,0,0,1
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,4,0,0,4,1,0,0,0,1
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,9,0,0,1,0,0,0,0,0


In [70]:
dataset = [image.split() for image in df_filtered['pixels'].tolist()]  # create full dataset of pixels, but each pixel is still a string

In [78]:
print(len(dataset),len(dataset[0]))  # shape is correct

35538 2304


In [79]:
# now do the pixel conversion to integer

for image in range(len(dataset)):
    dataset[image] = [int(pixel) for pixel in dataset[image]]

In [82]:
# create image dataset, properly shaped

dataset = np.array(dataset).reshape(-1, 48, 48, 1)

In [83]:
dataset.shape

(35538, 48, 48, 1)

In [ ]:
# MISSING: create a single function for all this